In [231]:
###IMPORT  LIBRARIES#####
import pandas as pd
import arcgis
from arcgis.features import GeoAccessor,GeoSeriesAccessor
from datetime import datetime
# these libraries would have to be downloaded from Nexus as I dont think they ship with ArcPro 2.5
import sqlalchemy
from sqlalchemy import create_engine
import cx_Oracle
import os
import arcpy
startTime = datetime.now()

#connect to studio
engine = create_engine('oracle://@P_PET_SAMP')
connection = engine.connect()
print(engine) 

#Setting Variables
arcpy.env.overwriteOutput = True
Studio_SDE = "I:\\UTCS\\Region\\SAmerica.Hou\\Ref_Data\\Guyana.hou\\General\\Maps\\Guyana_INT_to_Guyana_SDE\\HA_P_PET_SAMP.sde"
Project_gdb = "C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\MyProject.gdb"
splittatt = "C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\splittatt.gdb"

In [232]:
livetrace_query = ''' SELECT 
                dbxobj.name NAME,
                dbxobj.folder_path FOLDER_PATH,
                dbxobj.display_type Display_Type,
                dbxobj.domain_object Domain_Object,
                dbxobj.guid GUID,
                dbxobj.shape SHAPE,
                sdo_util.getnumelem(dbxobj.shape) NumElements,
                sdo_util.getnumvertices(dbxobj.shape) NumVertices

            FROM 
                GUYANA_INT.DBX_OBJECT dbxobj

            WHERE 
                GENERIC_DISPLAY_TYPE = 'Polygons' AND FOLDER_PATH LIKE '\Project\Input\Reference Datasets\Live Trace Outlines\%'  '''

In [309]:
#from sql query to pandas dataframe
polygons = pd.read_sql_query(sql = livetrace_query,con=connection)
polygons.columns = polygons.columns.str.upper()
polygons = polygons.drop(columns=["SHAPE"])
polygons.head()

,NAME,FOLDER_PATH,DISPLAY_TYPE,DOMAIN_OBJECT,GUID,NUMELEMENTS,NUMVERTICES
0,GY_CGG20_TC_4DBL_FT_KPSDM_EM19FWI_4x2_fl_3-36_cap_bb_avoq_zp_z,\Project\Input\Reference Datasets\Live Trace Outlines\Guyana,Polygons,DbxPolylineSet,E74ED15872A1460AAE219540599009A6,1,945
1,GY_CGG20_TC_4DBL_FT_KPSDM_EM19FWI_4x2_md_14-25_cap_bb_avoq_zp_z2t,\Project\Input\Reference Datasets\Live Trace Outlines\Guyana,Polygons,DbxPolylineSet,6B6235C5781C41DEB39EA531BB38FF63,101,1004
2,GY_CGG19_HH4DBL_FWI_2020CGG_1x1,\Project\Input\Reference Datasets\Live Trace Outlines\Guyana,Polygons,DbxPolylineSet,CFF570A97565403DB253B2CC79DE93CA,1,9
3,orinduik_2x8,\Project\Input\Reference Datasets\Live Trace Outlines\Guyana,Polygons,DbxPolylineSet,9C4956034D4948498EC6FD372A27CE0A,1,1894
4,GY_CGG19_TCE4DBL_FWI_2020CGG_1x1,\Project\Input\Reference Datasets\Live Trace Outlines\Guyana,Polygons,DbxPolylineSet,3BCE6844AC4646C1BEE0F4256672B223,1,12


In [310]:
Live_Trace_Layer = arcpy.MakeQueryLayer_management(input_database= Studio_SDE, out_layer_name="Live_Trace_Polylines", query=livetrace_query, oid_fields=["GUID"], shape_type="POLYLINE", srid="4326")

In [311]:
Live_trace_feature = arcpy.FeatureClassToFeatureClass_conversion(in_features= Live_Trace_Layer, out_path= Project_gdb, out_name="Live_Trace_3D_Outlines")

In [96]:
#one way to split records into separate features
#fields = ["NAME"]
#arcpy.analysis.SplitByAttributes(Live_trace_feature,splittatt,fields)

<Result 'C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\splittatt.gdb'>

In [312]:
arcpy.env.workspace = splittatt
arcpy.env.overwriteOutput = True
fields = ["SHAPE@","NAME","FOLDER_PATH","GUID","DISPLAY_TYPE","NUMELEMENTS","NUMVERTICES"]
with arcpy.da.SearchCursor(Live_trace_feature,fields) as cursor:
    for row in cursor:
        rowreplace = row[1].replace(" ","_")
        rowreplace = rowreplace.replace("-","")
        print(row)
        print(row[0])
        print(rowreplace)
        feature = arcpy.management.CopyFeatures(row[0],f"fc_{rowreplace}")
        feat2poly = arcpy.management.FeatureToPolygon(feature,f"{feature}_poly")
        field = "GUID"
        expression = row[3]
        print(expression)
        arcpy.AddField_management(feat2poly,field,"TEXT")
        arcpy.CalculateField_management(feat2poly,field,'"'+expression+'"',"PYTHON3")

merge_list = arcpy.ListFeatureClasses("*_poly")
polygon_studio = arcpy.management.Merge(merge_list,f"LiveTrace_studio_GUID")

(<Polyline object at 0x1caad20b240[0x1cac81247b0]>, 'GY_CGG20_TC_4DBL_FT_KPSDM_EM19FWI_4x2_fl_3-36_cap_bb_avoq_zp_z', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'E74ED15872A1460AAE219540599009A6', 'Polygons', 1.0, 945.0)
<geoprocessing describe geometry object object at 0x000001CAC81247B0>
GY_CGG20_TC_4DBL_FT_KPSDM_EM19FWI_4x2_fl_336_cap_bb_avoq_zp_z
E74ED15872A1460AAE219540599009A6
(<Polyline object at 0x1caad20bef0[0x1cb6d12e990]>, 'GY_CGG20_TC_4DBL_FT_KPSDM_EM19FWI_4x2_md_14-25_cap_bb_avoq_zp_z2t', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', '6B6235C5781C41DEB39EA531BB38FF63', 'Polygons', 101.0, 1004.0)
<geoprocessing describe geometry object object at 0x000001CB6D12E990>
GY_CGG20_TC_4DBL_FT_KPSDM_EM19FWI_4x2_md_1425_cap_bb_avoq_zp_z2t
6B6235C5781C41DEB39EA531BB38FF63
(<Polyline object at 0x1caad20b240[0x1cac81247b0]>, 'GY_CGG19_HH4DBL_FWI_2020CGG_1x1', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'CFF570A975

(<Polyline object at 0x1caad20b240[0x1cac81247b0]>, 'GY_CGG17_4DBL_FWI_2018CGG_4x4_aeps_tti_d', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'F7C292F4AF604328B933C06EE721B5EE', 'Polygons', 1.0, 6.0)
<geoprocessing describe geometry object object at 0x000001CAC81247B0>
GY_CGG17_4DBL_FWI_2018CGG_4x4_aeps_tti_d
F7C292F4AF604328B933C06EE721B5EE
(<Polyline object at 0x1caad20bef0[0x1cb6d12e990]>, 'GY_CGG17_4DBL_FWI_2018CGG_2x2', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', '9B18D8374F3A4DE485E94552B42D8126', 'Polygons', 1.0, 9.0)
<geoprocessing describe geometry object object at 0x000001CB6D12E990>
GY_CGG17_4DBL_FWI_2018CGG_2x2
9B18D8374F3A4DE485E94552B42D8126
(<Polyline object at 0x1caad20b240[0x1cac81247b0]>, 'GY_CGG17_4DBL_FWI_2018CGG_4x4_PAYARA_IP3_MODEL_2019_PPG', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'BB8FD2FA0E2642809DA7B93D20C4EC65', 'Polygons', 1.0, 6.0)
<geoprocessing describe geometry object object at 

(<Polyline object at 0x1caad20b240[0x1cac81247b0]>, 'liza_cgg13_1x2_akpstm_fr_16hz ', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'B94281BFC7C740AF844419F052D40F20', 'Polygons', 1.0, 1327.0)
<geoprocessing describe geometry object object at 0x000001CAC81247B0>
liza_cgg13_1x2_akpstm_fr_16hz_
B94281BFC7C740AF844419F052D40F20
(<Polyline object at 0x1caad20bef0[0x1cb6d12e990]>, 'liza_cgg13_1x2_fwi_xc_fr_xom ', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'EDE576A0018246BC81D4C042F9A6B9DD', 'Polygons', 1.0, 979.0)
<geoprocessing describe geometry object object at 0x000001CB6D12E990>
liza_cgg13_1x2_fwi_xc_fr_xom_
EDE576A0018246BC81D4C042F9A6B9DD
(<Polyline object at 0x1caad20b240[0x1cac81247b0]>, 'liza_cgg13_2x8', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'AC617A965574460E82BDBE2E61C3517A', 'Polygons', 1.0, 2502.0)
<geoprocessing describe geometry object object at 0x000001CAC81247B0>
liza_cgg13_2x8
AC617A965574460E82

(<Polyline object at 0x1caad20bef0[0x1cb6d12e990]>, 'gy_kP12_FT_cgg17_1x2_303_KP12_f25_36_IMP_AACS_AODS_GSPR_z', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'B4EB9C5C1277497284D9691912D37CF9', 'Polygons', 1.0, 3594.0)
<geoprocessing describe geometry object object at 0x000001CB6D12E990>
gy_kP12_FT_cgg17_1x2_303_KP12_f25_36_IMP_AACS_AODS_GSPR_z
B4EB9C5C1277497284D9691912D37CF9
(<Polyline object at 0x1caad20b240[0x1cac81247b0]>, 'gy_kP1_FT_cgg17_1x2_100_KP1_Raw_fl03_36_sshape2db_sec_IMP_diffusion_z', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', 'B21C30E39E964FF79550A90382C47AF3', 'Polygons', 131.0, 10053.0)
<geoprocessing describe geometry object object at 0x000001CAC81247B0>
gy_kP1_FT_cgg17_1x2_100_KP1_Raw_fl03_36_sshape2db_sec_IMP_diffusion_z
B21C30E39E964FF79550A90382C47AF3
(<Polyline object at 0x1caad20bef0[0x1cb6d12e990]>, 'gy_kP1_FT_cgg17_1x2_101_KP1_TTI_V0_crop', '\\Project\\Input\\Reference Datasets\\Live Trace Outlines\\Guyana', '

In [314]:
sdf = pd.DataFrame.spatial.from_featureclass(polygon_studio)


polygons.info()

final = pd.merge(sdf,polygons,how="left",left_on=["GUID"],right_on=["GUID"])
final.spatial.to_featureclass(location="C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\splittatt.gdb\polygons_final",overwrite = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   NAME           75 non-null     object
 1   FOLDER_PATH    75 non-null     object
 2   DISPLAY_TYPE   75 non-null     object
 3   DOMAIN_OBJECT  75 non-null     object
 4   GUID           75 non-null     object
 5   NUMELEMENTS    75 non-null     int64 
 6   NUMVERTICES    75 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 4.2+ KB


'C:\\Users\\srorta\\Desktop\\Guyana\\MyProject\\splittatt.gdb\\polygons_final'